In [1]:
import torch as np
from torchvision.utils import save_image
from torch.utils import data

from tools.celeba import CelebALoader
from dataset import Split

from models2.CSA import CSA
from tools.toml import load_option
from tools.mask import mask_iter

In [2]:
header_opt = load_option('options/header.toml')
loader = CelebALoader(header_opt.data_root)
dataset = Split(loader, header_opt.fine_size)

In [3]:
val_data = dataset.train(['attr'], header_opt.fine_size)

In [4]:
ty = ["attr", "identity", "bbox", "landmarks"]
val_data = dataset.train(ty)
val_data

Dataset VisionDataset
    Number of datapoints: 162770
    Root keys: dict_keys(['splits', 'identity', 'bbox', 'landmarks_align', 'attr', 'Z'])
    Target type: ['attr', 'identity', 'bbox', 'landmarks']
    Split: train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [6]:
val_data = dataset.valid('bbox')
valset = data.DataLoader(val_data,
                         batch_size=header_opt.batch_size,
                         shuffle=True)
maskset = mask_iter(header_opt.mask_root, header_opt.fine_size)
print('数据集数量：', len(valset))
print('掩码数量：', len(maskset))

训练集数量： 19867
掩码数量： 12000


In [7]:
import time
import torch
import torchvision
base_opt = load_option('options/base.toml')
opt = load_option('options/train.toml')
opt.update(base_opt)

model = CSA(**opt)

initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorCSA(
  (model): UnetSkipConnectionBlock_3(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3(
            (model): Sequential(
              (0): LeakyReLU(negat

In [8]:
load_name = "14-1"
model.load(load_name)

In [9]:
import time
from pathlib import Path

In [10]:
def mask_op(mask):
    mask = mask.cuda()
    mask = mask[0][0]
    mask = torch.unsqueeze(mask, 0)
    mask = torch.unsqueeze(mask, 1)
    mask = mask.byte()
    return mask

In [11]:
save_dir = '../measure/true'
save_dir = Path(save_dir)
if not save_dir.exists():
    save_dir.mkdir(parents=True, exist_ok=True)

epoch = 1
i = 0
total_steps = 0
for batch, mask in zip(valset, maskset):
    iter_start_time = time.time()
    image = batch[0]
    image = image.cuda()
    mask = mask_op(mask)
    model.set_input(image, mask)
    model.set_gt_latent()
    model.test()
    real_A, real_B, fake_B = model.get_current_visuals()
    total_steps += header_opt.batch_size
    pic = (torch.cat([real_A, real_B, fake_B], dim=0) + 1) / 2.0
    torchvision.utils.save_image(pic, f"{save_dir}/epoch{epoch}-{total_steps}.jpg", nrow=1)

KeyboardInterrupt: 